# Beaking down the code

In [2]:
%load_ext autoreload
%autoreload 2

## The `AbstractParser`  class

The `AbstractParser` class gives us a template to be used in the parsers. The functions that need to be defined are:
* filename(self, date: datetime, **kwargs): return the filename, given a date
* foldername(self, date: datetime, **kwargs): return the folder, given a date

Additionally, the following information are also necessary as `constants` dictionary:
* root - The root folder, where to find/store the files
* var - The variable name (to be extracted from the dataset)
* name - The name to be displayed in the system

Here is a simple example of a parser implementation:


In [1]:
from mergedownloader.parser import AbstractParser
from datetime import datetime

class Parser(AbstractParser):
    constants = {
        'root': "/files/GPM",
        'var': 'precipitation',
        'name': 'Daily precipitation'
    }

    def filename(self, date: datetime, **__):
        return f'Prec_{date:%Y%m%d}.nc'
    
    def foldername(self, date: datetime, **__):
        return 'DailyPrec'


In [3]:
# now, let's see our parser in action
DATE = datetime(2022, 1, 1)

parser = Parser()

print(parser.filename(DATE))
print(parser.foldername(DATE))
print(parser.remote_folder(DATE))
print(parser.remote_target(DATE))
print(parser.local_folder(DATE, '/tmp'))


Prec_20220101.nc
DailyPrec
/files/GPM/DailyPrec
/files/GPM/DailyPrec/Prec_20220101.nc
/tmp/DailyPrec


## Inpe Parsers Classes

The GPM (Global Precipitation Measurement) files for the South America are processed by the INPE (Brazilian Spatial Institute). The values are adjusted by combining TRMM and surface observations from ground stations, through the MERGE model (Rozante et al., 2010).

The `inpeparser` module has the parsers to work with the INPE structure. The `*` will import the parsers and corresponding DataTypes.

Reference
* Rozante et al., 2010 - https://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/Rozante_et.al.2010.pdf

In [4]:
from mergedownloader.inpeparser import *

In [5]:
for dt in InpeTypes:
    print(dt, dt.value)

InpeTypes.DAILY_RAIN 1
InpeTypes.MONTHLY_ACCUM_YEARLY 2
InpeTypes.DAILY_AVERAGE 3
InpeTypes.MONTHLY_ACCUM 4
InpeTypes.MONTHLY_ACCUM_MANUAL 5
InpeTypes.YEARLY_ACCUM 6
InpeTypes.HOURLY_WRF 7


In [6]:
DATE = datetime(2022, 1, 1)

parser = YearAccumulatedParser()

print(parser.filename(DATE))
print(parser.foldername(DATE))
print(parser.remote_folder(DATE))
print(parser.remote_target(DATE))
print(parser.local_folder(DATE, '/tmp'))


MERGE_CPTEC_acum_2022.nc
YEAR_ACCUMULATED
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/YEAR_ACCUMULATED
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/YEAR_ACCUMULATED/MERGE_CPTEC_acum_2022.nc
/tmp/YEAR_ACCUMULATED


In [7]:
parser = MonthlyAccumParser()

print(parser.filename(DATE))
print(parser.foldername(DATE))
print(parser.remote_folder(DATE))
print(parser.remote_target(DATE))
print(parser.local_folder(DATE, '/tmp'))


MERGE_CPTEC_acum_jan.nc
MONTHLY_ACCUMULATED
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/MONTHLY_ACCUMULATED
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/MONTHLY_ACCUMULATED/MERGE_CPTEC_acum_jan.nc
/tmp/MONTHLY_ACCUMULATED


In [8]:
parser = DailyAverageParser()

print(parser.filename(DATE))
print(parser.foldername(DATE))
print(parser.remote_folder(DATE))
print(parser.remote_target(DATE))
print(parser.local_folder(DATE, '/tmp'))


MERGE_CPTEC_12Z01jan.nc
DAILY_AVERAGE
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/DAILY_AVERAGE
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/DAILY_AVERAGE/MERGE_CPTEC_12Z01jan.nc
/tmp/DAILY_AVERAGE


In [9]:
parser = DailyParser()

print(parser.filename(DATE))
print(parser.foldername(DATE))
print(parser.remote_folder(DATE))
print(parser.remote_target(DATE))
print(parser.local_folder(DATE, '/tmp'))


MERGE_CPTEC_20220101.grib2
DAILY/2022/01
/modelos/tempo/MERGE/GPM/DAILY/2022/01
/modelos/tempo/MERGE/GPM/DAILY/2022/01/MERGE_CPTEC_20220101.grib2
/tmp/DAILY/2022/01


In [10]:
parser = MonthlyAccumYearlyParser()

print(parser.filename(DATE))
print(parser.foldername(DATE))
print(parser.remote_folder(DATE))
print(parser.remote_target(DATE))
print(parser.local_folder(DATE, '/tmp'))


MERGE_CPTEC_acum_jan_2022.nc
MONTHLY_ACCUMULATED_YEARLY
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/MONTHLY_ACCUMULATED_YEARLY
/modelos/tempo/MERGE/GPM/CLIMATOLOGY/MONTHLY_ACCUMULATED_YEARLY/MERGE_CPTEC_acum_jan_2022.nc
/tmp/MONTHLY_ACCUMULATED_YEARLY


To make things easier, we also define a list of parser instances that can be used with the `Downloader`.

In [84]:
from mergedownloader.inpeparser import *

In [82]:
del InpeParsers

In [85]:
InpeParsers

{<InpeTypes.DAILY_RAIN: 1>: Instance of DailyParser,
 <InpeTypes.MONTHLY_ACCUM_YEARLY: 2>: Instance of MonthlyAccumYearlyParser,
 <InpeTypes.DAILY_AVERAGE: 3>: Instance of DailyAverageParser,
 <InpeTypes.MONTHLY_ACCUM: 4>: Instance of MonthlyAccumParser,
 <InpeTypes.MONTHLY_ACCUM_MANUAL: None>: None,
 <InpeTypes.YEARLY_ACCUM: 5>: Instance of YearAccumulatedParser}

In [52]:
InpeParsers[InpeTypes.DAILY_AVERAGE]

Instance of DailyAverageParser